<a href="https://colab.research.google.com/github/Itsuki966/docker-selenium-sample/blob/main/id_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import japanize_matplotlib
import geopandas as gpd
from sklearn.cluster import KMeans

# データを読み込む

In [156]:
# データを読み込むための関数
def load_data(year):
  data = pd.read_excel("city_data.xlsx", sheet_name = str(year), index_col=0)
  return data

In [230]:
data_2000 = load_data(2000)
data_2005 = load_data(2005)
data_2010 = load_data(2010)
data_2015 = load_data(2015)
data_list = [data_2000, data_2005, data_2010, data_2015]

# データの前処理

In [231]:
# 欠損値を削除する関数
def preprocessing(data):
  data.dropna(inplace=True)
  data.replace("-", 0, inplace=True)
  data.reset_index(drop=True, inplace=True)

def add_next_year_population(data, data_next):
  temp = data_next[["code","総人口"]]
  temp.rename(columns={"総人口":"次の総人口"}, inplace=True)
  data = pd.merge(data, temp, on="code", how="inner")
  return data

def calc_population_rate(data):
  data["前年比人口"] = data["次の総人口"] / data["総人口"]

In [ ]:
for d in data_list:
  preprocessing(d)

for i in range(len(data_list)-1):
  data_list[i] = add_next_year_population(data_list[i], data_list[i+1])
  calc_population_rate(data_list[i])

In [233]:
# 全てのデータを結合 (教師あり学習で利用するデータを作成)
conbined_data = pd.concat(data_list[:3])
conbined_data.reset_index(drop=True, inplace=True)

# 教師なし学習
市町村の分類をしてみよう


## 利用できるデータ項目
*   予算関係のデータ
> '歳出決算額', '社会福祉費', '老人福祉費', '児童福祉費', '農林水産業費', '商工費', '都市計画費', '住宅費', '財政力指標'

*   教育関連のデータ
> '教育費', '小学校費', '中学校費', '高校費', '幼稚園費', '小学校数', '小学校教員数', '中学校数', '中学校教員数', '高校数'
*   医療関連のデータ
> '介護老人福祉施設(65歳以上人口10万人当たり)', '一般病院数/10万人', '一般診療所数/10万人', '一般病院数/可住地面積', '一般診療所数/可住地面積'
*   人口データ
> '自市区町村で従業・通学している人口', '流出人口（県内他市区町村で従業・通学している人口）',
'流出人口（他県で従業・通学している人口）', '流入人口（県内他市区町村に常住している人口）', '流入人口（他県に常住している人口）', '若年層人口', '総人口', '昼夜間人口比率(%)'
*   産業構造データ
> '第一次産業就業者', '第二次産業就業者', '第三次産業就業者'











In [ ]:
# 分析対象データ（2000年のデータは"0"、2005年は"1"、2010年は"2"、2015年は"3"）
o_data = data_list[3]

# 利用するデータ項目を指定
cols = ['第二次産業就業者', '第一次産業就業者']

o_data = o_data[cols]
o_data["city"] = list(data_list[3]["area"])
o_data["code"] = list(data_list[3]["code"].astype("str"))

In [ ]:
wcss = []
for i in range(1, 11):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
  kmeans.fit(o_data[cols])
  wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title("エルボー法")
plt.xlabel("クラスタ数")
plt.ylabel("WCSS")
plt.show()
# 肘に相当する部分がクラスタ数

In [ ]:
# cluster_numをエルボー法の結果に応じて指定
cluster_num = 3
kmeans = KMeans(n_clusters=cluster_num, init='k-means++', max_iter=300, n_init=10, random_state=0)
result = kmeans.fit_predict(o_data[cols])

o_data["cluster"] = result

plt.scatter(o_data[cols[0]], o_data[cols[1]], c=result, cmap='rainbow')
plt.xlabel(cols[0])
plt.ylabel(cols[1])
plt.show()

In [ ]:
# 各クラスタにどのような市町村が分類されているのかを表示
for c in range(cluster_num):
  print(f"cluster {c}")
  print(list(o_data[o_data["cluster"] == c]["city"]))
  print("")

## 日本地図に落とし込む (ビミョー)

In [115]:
map = gpd.read_file("N03-18_180101.shp", encoding="cp932")

In [116]:
def add_zero(code):
  if len(code) == 4:
    return "0" + code
  else:
    return code

In [119]:
o_data["code"] = o_data["code"].apply(add_zero)

temp_city = list(map[map["N03_004"].str.contains("区")]["N03_003"])
temp_city = list(set(temp_city))
temp_city.remove(None)

for i in temp_city:
  new_code = o_data[o_data["city"].str.contains(i)]["code"]
  indexes = list(map[map["N03_003"]==i].index)
  map.loc[list(map[map["N03_003"]==i].index), "N03_007"] = new_code

japan_map = map.copy()
japan_map.drop(["N03_001","N03_002"], axis=1, inplace=True)
japan_map.rename(columns={"N03_007":"code"}, inplace=True)

plot_data = pd.merge(japan_map, o_data, on="code", how='inner')

In [ ]:
# マップを表示する
plot_data.plot(
    column="cluster",
    cmap="rainbow",
    legend=True,
)

# 教師あり学習
2020年の前年比人口を予測してみよう

In [146]:
!pip install lightgbm

In [202]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [239]:
# 目的変数
y = conbined_data["前年比人口"]

# 説明変数
x = conbined_data.drop(["year","code","area","次の総人口","前年比人口"], axis=1)

# データを訓練用とテスト用に分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
model = lgb.LGBMRegressor()

# 学習データを利用したモデルの学習
model.fit(x_train, y_train)

# モデルの評価
mse = mean_squared_error(y_test, model.predict(x_test))

print("")
print("MSEが小さければ小さいほどモデルの精度が高いことを示す")
print(f"MSE: {mse}")

## 学習済みのモデルと2015年のデータを利用して2020年の前年比人口を予測する

In [256]:
# 2015年のデータを用意
data_2015["昼夜間人口比率(%)"] = data_2015["昼夜間人口比率(%)"].astype("float")
x_2015 = data_2015.drop(["year","code","area"], axis=1)

# 予測
pred_2015_y = model.predict(x_2015)

In [ ]:
# 結果の表示
temp = zip(list(data_2015["area"]), pred_2015_y)
sorted(temp, key=lambda x:x[1], reverse=True)